# Prepare

先讓結巴分詞套用繁體字典<br>
讓之後斷詞表現的好一些

In [1]:
import jieba
import re
import pandas as pd
import time

jieba.set_dictionary('./dict.txt.big.txt')
jieba.initialize()

Building prefix dict from /home/tonylee/2018DSAI/dsai2018_hw2/dict.txt.big.txt ...
Dumping model to file cache /tmp/jieba.u78196fc27356e93ab4692506d3f4c643.cache
Loading model cost 2.053 seconds.
Prefix dict has been built succesfully.


# Source Traversal

目前 source 擁有 10000 筆資料<br>
因此拜訪的方法就很重要

在這裡發現 pandas 的 `iterrows()` 的表現非常不好<br>
反而使用 list 搭配 `enumerate()` 能有非常好的表現

時間上相差幾乎百倍以上

我剛開始也是使用 `iterrows()` , 時間上真的慢到懷疑人生

In [2]:
source = pd.read_csv('./source.csv', header=None)
source.head()

,0,1
0,1,嘎嫂新髮型宛如高中生 蔡阿嘎合照根本「誘拐未成年少女」
1,2,美國報告未提證據 俄：操縱大選是無中生有
2,3,新年搶優惠 通路Apple商品加碼送
3,4,台灣癌症時鐘加快 每5分26秒就1人罹癌
4,5,藍營胡批兩岸急凍壓垮興航 綠委回嗆「不用腦子的政黨」


In [3]:
%%time

for index,row in source.iterrows():
    title = row[1]
    #terms = splitKeyWord(title)
    #terms = jieba.cut_for_search(title)
    continue

CPU times: user 7 s, sys: 6.42 ms, total: 7.01 s
Wall time: 7.01 s


In [4]:
%%time

sourceData = source.iloc[:,1].values

for index, title in enumerate(sourceData):
    #terms = splitKeyWord(title)
    #terms = jieba.cut_for_search(title)
    continue

CPU times: user 21.6 ms, sys: 0 ns, total: 21.6 ms
Wall time: 24.3 ms


# Split title

這裡除了結巴分詞<br>
在同學的建議下, 另外用正規表示式 `RE` 撰寫了兩種分出詞的方法

- `splitKeyWord`

先將單個字一個個抓出來, 然後從 1 個字到 n 個字分別組合成新的字串
成為 query 用的 keyword

算是暴力找出長度 n 個字以下的所有組合

- `splitKeyWordByRE`

與前個方法類似, 但使用 `RE` 本身去取代 for 的遞迴合成字串
以加速找出 keyword 的速度

---

最後是 `splitKeyWordByRE` 快於 `splitKeyWord` 與 `jieba`(結巴)

`jieba` 的好處在於只有找到有意義的 keyword, 可是為了找到有意義的單詞需要去比對字典<br>
因此需要花費較多時間, 犧牲掉了效能

加上該作業需要 100% 正確, 為了保險起見, 選擇 `splitKeyWordByRE`

In [5]:
def splitKeyWord(str, n):
    regex = r"[\u4e00-\ufaff]|[0-9]+|[a-zA-Z]+\'*[a-z]*|[^0-9]"
    matches = re.findall(regex, str, re.UNICODE)
    words = list()
    for j in range(n):
        words.append([])
        for i in range(len(matches) - j):
            words[j].append(''.join(matches[i:i+j+1]))
    return words

def splitKeyWordByRE(str,n):
    words = [re.findall(r"[\u4e00-\ufaff]{1}|[0-9]+|[a-zA-Z]+\'*[a-zA-Z]*", str, re.UNICODE)]        
    for i in range(2,n+1):
        words.append(re.findall(r"(?=([\u4e00-\ufaff]{%d}))" % (i), str, re.UNICODE))
    return words

In [6]:
import random
test_sentence = source.iloc[random.randint(0,len(source)-1),1]
print(test_sentence,'\n')
print("using split n word by RE\n\n",splitKeyWordByRE(test_sentence,3), "\n")
print("using split n word\n\n",splitKeyWord(test_sentence,3), "\n")
print("using jieba\n\n",jieba.lcut_for_search(test_sentence), "\n")

台中、高雄下午反空污大遊行　訴求是這些 

using split n word by RE

 [['台', '中', '高', '雄', '下', '午', '反', '空', '污', '大', '遊', '行', '訴', '求', '是', '這', '些'], ['台中', '高雄', '雄下', '下午', '午反', '反空', '空污', '污大', '大遊', '遊行', '訴求', '求是', '是這', '這些'], ['高雄下', '雄下午', '下午反', '午反空', '反空污', '空污大', '污大遊', '大遊行', '訴求是', '求是這', '是這些']] 

using split n word

 [['台', '中', '、', '高', '雄', '下', '午', '反', '空', '污', '大', '遊', '行', '\u3000', '訴', '求', '是', '這', '些'], ['台中', '中、', '、高', '高雄', '雄下', '下午', '午反', '反空', '空污', '污大', '大遊', '遊行', '行\u3000', '\u3000訴', '訴求', '求是', '是這', '這些'], ['台中、', '中、高', '、高雄', '高雄下', '雄下午', '下午反', '午反空', '反空污', '空污大', '污大遊', '大遊行', '遊行\u3000', '行\u3000訴', '\u3000訴求', '訴求是', '求是這', '是這些']] 

using jieba

 ['台', '中', '、', '高雄', '下午', '反空', '污大', '遊行', '\u3000', '訴求', '是', '這些'] 



In [7]:
%%time

sourceData = source.iloc[:,1].values

for index, title in enumerate(sourceData):
    terms = splitKeyWordByRE(title,3)


CPU times: user 1.62 s, sys: 4.35 ms, total: 1.63 s
Wall time: 1.63 s


In [8]:
%%time

sourceData = source.iloc[:,1].values

for index, title in enumerate(sourceData):
    terms = splitKeyWord(title,3)


CPU times: user 3.04 s, sys: 12.3 ms, total: 3.05 s
Wall time: 3.05 s


In [9]:
%%time

sourceData = source.iloc[:,1].values

for index, title in enumerate(sourceData):
    terms = jieba.lcut_for_search(title)


CPU times: user 16.8 s, sys: 550 µs, total: 16.8 s
Wall time: 16.8 s


# Search engine

這邊使用 python 本身具有的 `dictionary` 資料結構<br>
是使用 key 經過 hash 得到 index, 藉此加入查找 value 的速度<br>
也就是常見的 hash table

但這樣的 hash table 會有碰撞的 issue<br>
本來想說不同長度的詞分開存進不同的 `dictionary`<br>
希望減少碰撞進而加速 insert 的動作

但發現全部放進同一個 `dictionary` 也不會造成效能太大的損耗<br>
而且有利於後面要查詢的時候不用去判斷要使用哪一個 `dictionary`

因此實作選擇使用 `putAll()` and `queryAll()` 這兩個

而在這個引擎中使用 keyword 當作 key, 文章標題的索引集合當作 value<br>
這樣可以隨意丟入 keyword 查詢到有哪些文章標題含有這個 keyword

In [10]:
def English(str):
    regex = r"[a-zA-Z]+\'*[a-z]*"
    matches = re.findall(regex, str, re.UNICODE)
    if matches:
        return True
    else:
        return False

class searchEngine:
    def __init__(self, n):
        self.dicts = []
        self.dictAll = dict()
        for i in range(n):
            self.dicts.append(dict())
    
    def put(self, n, word, index):
        if n >= len(self.dicts):
            return
        if word not in self.dicts[n]:
            self.dicts[n][word] = set([index])
        else:
            self.dicts[n][word].add(index)
    
    def putAll(self, word, index):
        if word not in self.dictAll:
            self.dictAll[word] = set([index])
        else:
            self.dictAll[word].add(index)
    
    def query(self, word):
        if English(word):
            return self.dicts[0].get(word, set())
        else:
            return self.dicts[len(word) - 1].get(word, set())
    
    def queryAll(self, word):
        return self.dictAll.get(word, set())

n_words = 3
engine = searchEngine(n_words)

In [11]:
%%time

sourceData = source.iloc[:,1].values

for index, title in enumerate(sourceData):
    terms = splitKeyWordByRE(title,n_words)
    for i in range(n_words):
        for term in terms[i]:
            engine.put(i, term, index+1)

CPU times: user 7.79 s, sys: 220 ms, total: 8.01 s
Wall time: 8.01 s


In [12]:
%%time

sourceData = source.iloc[:,1].values

for index, title in enumerate(sourceData):
    terms = splitKeyWordByRE(title,n_words)
    for i in range(n_words):
        for term in terms[i]:
            engine.putAll(term, index+1)

CPU times: user 7.52 s, sys: 168 ms, total: 7.69 s
Wall time: 7.69 s


In [13]:
query = "MLB"

a = engine.query(query)

b = engine.queryAll(query)

print(a == b)

True


# Query

來到關鍵的地方了, 前面利用 `dictionary` 建立好搜尋用的引擎

這邊就是要來 query 這個引擎了

首先使用 `parseQuery` 去分析哪些是要 query 的 keyword , 哪些是要執行的集合操作

那因為作業要求有提到, 一條 query 只會有同一種操作

因此把第一個 keyword 的集合找出來, 後面遇到的集合都與第一個集合使用該集合操作

最後剩下就會是該 query 的答案

轉成 `list` 進行 `sort()`, 並使用 `join()` 變成字串就完成作業要求的答案

In [16]:
def parseQuery(query):
    if query[-1] == '\n':
        query = query[:-1]
    words = query.split(" ")
    op = None
    queryWord = []
    for word in words:
        if (word == "and") or (word == "or") or (word == "not"):
            op = word
            continue
        queryWord.append(word)
    return [queryWord, op]

with open('./query.txt', 'r') as queryfile:
    querys = queryfile.readlines()

In [17]:
print(parseQuery(querys[random.randint(0,len(querys)-1)]))

[['NBA', '大三元'], 'and']


In [18]:
%%time

answer1 = []

for query in querys:
    keywords, op = parseQuery(query)
    
    answer = engine.queryAll(keywords[0])
    for keyword in keywords[1:]:
        if op == "and":
            answer = answer & engine.queryAll(keyword)
        if op == "or":
            answer = answer | engine.queryAll(keyword)
        if op == "not":
            answer = answer - engine.queryAll(keyword)
    
    answer = list(answer)
    answer.sort()
    if len(answer) == 0:
        answer = '0'
    else:
        answer = ','.join(str(num) for num in answer)
    #print(query,"\n",answer,"\n")
    answer1.append(answer)
    

CPU times: user 9.43 ms, sys: 31 µs, total: 9.46 ms
Wall time: 9.4 ms


In [19]:
%%time

answer2 = []

for query in querys:
    keywords, op = parseQuery(query)
    
    answer = engine.query(keywords[0])
    for keyword in keywords[1:]:
        if op == "and":
            answer = answer & engine.query(keyword)
        if op == "or":
            answer = answer | engine.query(keyword)
        if op == "not":
            answer = answer - engine.query(keyword)
    
    answer = list(answer)
    answer.sort()
    if len(answer) == 0:
        answer = '0'
    else:
        answer = ','.join(str(num) for num in answer)
    #print(query,"\n",answer,"\n")
    answer2.append(answer)
    

CPU times: user 5.56 ms, sys: 0 ns, total: 5.56 ms
Wall time: 5.38 ms


In [20]:
for i in range(len(querys)):
    print(querys[i], answer1[i] == answer2[i])

電玩 and 宅男
 True
美國 and 北韓
 True
美國 or 台灣 or 中國
 True
母親節 and 禮物
 True
職籃 or 職棒
 True
台灣 and GDP
 True
NBA or MLB
 True
NBA and 大三元
 True


# Reference

- 華哥
- 翁帥哥
- [使用 Python 模块 re 实现解析小工具](https://www.ibm.com/developerworks/cn/opensource/os-cn-pythonre/index.html)
- [Python字典对象实现原理](https://foofish.net/python_dict_implements.html)